In [2]:
import pandas as pd
genuine_accounts_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/genuine_accounts.csv/tweets.csv', low_memory=False)
traditional_spambots_1_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/traditional_spambots_1.csv/tweets.csv', low_memory=False)
fake_followers_tweets=pd.read_csv('cresci-2017.csv/datasets_full.csv/fake_followers.csv/tweets.csv', low_memory=False)
social_spambots_3_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_3.csv/tweets.csv', low_memory=False)
social_spambots_2_tweets = pd.read_csv('cresci-2017.csv/datasets_full.csv/social_spambots_2.csv/tweets.csv', low_memory=False)
russian_troll_tweets = pd.read_csv('russian-troll-tweets/tweets.csv', low_memory=False)

In [438]:
a = genuine_accounts_tweets
b= russian_troll_tweets

In [440]:
a['label']=int(0)
b['label']=int(1)

import random
a1 = a.sample(n=1000, random_state=1)
a2 = a.drop(index=a1.index).sample(n=1000, random_state=0)
b1 = b.sample(n=1000, random_state=1)
b2 = b.drop(index=b1.index).sample(n=1000, random_state=0)
b3=b.drop(index=b1.index).drop(index=b2.index).sample(n=1000, random_state=0)

In [441]:
ab=pd.concat([b1, a1, b3], axis=0)
ab_text = ab[['text', 'label']]
ab_text = ab_text.fillna('None')

a2b2=pd.concat([b2, a2], axis=0)
a2b2_text=a2b2[['text', 'label']]
a2b2_text = a2b2_text.fillna('None')

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/Users/xinchunli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [442]:
# clean content: 
from spacy.lang.en.stop_words import STOP_WORDS
import re
import os
def cleanContent(contents):
    contents_new=[]
    contents_list = contents.tolist()
    tweets_list_new=[]
    for tweet in contents_list:
        if 'http' in tweet:
            index = tweet.rfind('http')
            tweet = tweet.replace(tweet[index:], '')
        tweets_list_new.append(tweet)
    for content in tweets_list_new:
        # Convert to lowercase
        content_low=content.lower()
        # Remove punctuation and any other non-alphabet characters
        content_low_pnt = re.sub(r'[^\w\s]', '', content_low)
        content_low_pnt = content_low_pnt.replace(os.linesep, " ")
        content_low_pnt = content_low_pnt.replace("\t", " ")
        content_low_pnt_alpha = re.sub('[^A-Za-z\s]', '', content_low_pnt)
        # Remove stopwords
        words=content_low_pnt_alpha.split()
        no_stop_words=[word for word in words if word not in STOP_WORDS]
        content=' '.join(no_stop_words)
        contents_new.append(content)
    return contents_new

In [443]:
ab_text.text=cleanContent(ab_text.text)
a2b2_text.text=cleanContent(a2b2_text.text)

In [444]:
ab_text_list=[]
for text in ab_text.text:
    ab_text_list.append(text)

In [445]:
a2b2_text_list=[]
for text in a2b2_text.text:
    a2b2_text_list.append(text)

In [446]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# SVM

In [447]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', SVC(gamma='auto', max_iter=100))])

text_clf.fit(ab_text_list, ab_text.label)  

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
  max_iter=100, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [448]:
import numpy as np
predicted = text_clf.predict(a2b2_text_list)
np.mean(predicted == a2b2_text.label)            

0.5201125

In [449]:
from sklearn import metrics
print(metrics.classification_report(a2b2_text.label, predicted))

metrics.confusion_matrix(a2b2_text.label, predicted)

              precision    recall  f1-score   support

           0       0.62      0.11      0.18     80000
           1       0.51      0.93      0.66     80000

   micro avg       0.52      0.52      0.52    160000
   macro avg       0.56      0.52      0.42    160000
weighted avg       0.56      0.52      0.42    160000



array([[ 8524, 71476],
       [ 5306, 74694]])

# Random Forest

In [450]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=20, criterion='entropy',
                                   random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [451]:
import numpy as np
predicted = text_clf.predict(a2b2_text_list)
np.mean(predicted == a2b2_text.label)            

0.504375

In [452]:
from sklearn import metrics
print(metrics.classification_report(a2b2_text.label, predicted))

metrics.confusion_matrix(a2b2_text.label, predicted)

              precision    recall  f1-score   support

           0       1.00      0.01      0.02     80000
           1       0.50      1.00      0.67     80000

   micro avg       0.50      0.50      0.50    160000
   macro avg       0.75      0.50      0.34    160000
weighted avg       0.75      0.50      0.34    160000



array([[  700, 79300],
       [    0, 80000]])

# Gradient Boosting

In [453]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])

In [454]:
import numpy as np
predicted = text_clf.predict(a2b2_text_list)
np.mean(predicted == a2b2_text.label)            

0.84773125

In [455]:
from sklearn import metrics
print(metrics.classification_report(a2b2_text.label, predicted))

metrics.confusion_matrix(a2b2_text.label, predicted)

              precision    recall  f1-score   support

           0       0.89      0.79      0.84     80000
           1       0.81      0.90      0.86     80000

   micro avg       0.85      0.85      0.85    160000
   macro avg       0.85      0.85      0.85    160000
weighted avg       0.85      0.85      0.85    160000



array([[63289, 16711],
       [ 7652, 72348]])

# Logistic Regression

In [456]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(penalty="l1",random_state=0))])

text_clf.fit(ab_text_list, ab_text.label)  

/Users/xinchunli/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...e, penalty='l1', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [457]:
import numpy as np
predicted = text_clf.predict(a2b2_text_list)
np.mean(predicted == a2b2_text.label)            

0.87690625

In [458]:
from sklearn import metrics
print(metrics.classification_report(a2b2_text.label, predicted))

metrics.confusion_matrix(a2b2_text.label, predicted)

              precision    recall  f1-score   support

           0       0.90      0.85      0.87     80000
           1       0.86      0.90      0.88     80000

   micro avg       0.88      0.88      0.88    160000
   macro avg       0.88      0.88      0.88    160000
weighted avg       0.88      0.88      0.88    160000



array([[68226, 11774],
       [ 7921, 72079]])